In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('dataset/melb_data.csv')

In [3]:
data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [4]:
y = data.Price
X = data.drop(['Price'], axis=1)

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, test_size=0.8, random_state=0)

In [5]:
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]

In [6]:
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [7]:
numerical_cols[:5]

['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom']

In [8]:
my_cols = categorical_cols + numerical_cols

In [9]:
len(my_cols)

15

In [10]:
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [11]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
8997,h,S,Western Metropolitan,3,10.4,3042.0,3.0,1.0,1.0,773.0,NaN,NaN,-37.72265,144.87930,3464.0
11264,h,S,Western Metropolitan,4,11.0,3018.0,4.0,1.0,1.0,499.0,NaN,NaN,-37.86639,144.82625,5301.0
5974,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,2.0,487.0,NaN,NaN,-37.78830,144.84390,3755.0
8333,h,PI,Northern Metropolitan,2,2.6,3121.0,2.0,1.0,0.0,134.0,63.7,1900.0,-37.81700,145.00580,14949.0
2910,h,S,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,4.0,704.0,102.0,1960.0,-37.70210,144.91360,8870.0


# Preprocessing Steps

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [13]:
numerical_transformer = SimpleImputer(strategy='constant')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data


In [14]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model

## With the pipeline, we preprocess the training data and fit the model in a single line of code. (In contrast, without a pipeline, we have to do imputation, one-hot encoding, and model training in separate steps. This becomes especially messy if we have to deal with both numerical and categorical variables!)

## With the pipeline, we supply the unprocessed features in X_valid to the predict() command, and the pipeline automatically preprocesses the features before generating predictions. (However, without a pipeline, we have to remember to preprocess the validation data before making predictions.)

In [15]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=0)

### Bundle preprocessingand modeling code in a pipeline

In [16]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
    ('model', model)])

In [18]:
pipeline.fit(X_train, y_train)


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='constant',
                                                                verbose=0),
                                                  ['Rooms', 'Distance',
                                                   'Postcode', 'Bedroom2',
                                                   'Bathroom', 'Car',
 

In [20]:
preds = pipeline.predict(X_valid)

In [21]:
score = mean_absolute_error(y_valid, preds)
print('MAE', score)

MAE 178765.02650229682
